In [21]:
import yfinance as yf
import pandas as pd
import numpy as np
import math
import plotly.express as px 
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pypfopt
from pypfopt import risk_models, expected_returns, plotting
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import objective_functions
from pypfopt.plotting import plot_weights
from scipy.stats import ttest_ind
import copy
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import SimpleRNN,LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
import random
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from pandas_datareader.famafrench import get_available_datasets
import pandas_datareader.data as web
import statsmodels.api as sm

In [22]:
tickers = ["BIIB","LRCX","RHI", "WSM","BIO","^GSPC"] 

In [23]:
ohlc = yf.download(tickers, start="2022-02-28")

[*********************100%***********************]  6 of 6 completed


In [24]:
prices = ohlc["Adj Close"].dropna(how="all")
daily = prices[:-23].pct_change()
daily.dropna(inplace=True)
daily['Portfolio Return']=daily['BIIB']*0.08+daily['BIO']*0.10+daily['LRCX']*0.075+daily['RHI']*0.285+daily['WSM']*0.05
Return=list(daily['Portfolio Return'])

In [25]:
daily.head(5)

,BIIB,BIO,LRCX,RHI,WSM,^GSPC,Portfolio Return
Date,,,,,,,
2022-03-01,-0.003175,-0.014713,-0.037018,-0.055865,0.012771,-0.015474,-0.019785
2022-03-02,-0.012123,-0.007896,0.024881,0.034428,0.020721,0.018643,0.010955
2022-03-03,0.016555,-0.005132,-0.018971,0.005192,0.012755,-0.005255,0.001506
2022-03-04,-0.008427,-0.043040,-0.032621,-0.009061,0.001055,-0.007934,-0.009954
2022-03-07,-0.017522,-0.055584,-0.069668,-0.055290,-0.068436,-0.029518,-0.031365


In [26]:
len(get_available_datasets())
FF = web.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench', start='2022-03-01', end='2022-05-30')
print(FF['DESCR'])
data = FF[0]
data = data.dropna()
data = data/100 
data['Port_Ret']=Return
FF_5 = pd.DataFrame(data)
FF_5['RP-Rf']=FF_5['Port_Ret']-FF_5['RF']
X = FF_5[['Mkt-RF','SMB','HML','RMW','CMA']]
Y = FF_5['RP-Rf']
X = sm.add_constant(X)

F-F Research Data 5 Factors 2x3 daily
-------------------------------------

This file was created by CMPT_ME_BEME_OP_INV_RETS_DAILY using the 202205 CRSP database. The 1-month TBill return is from Ibbotson and Associates, Inc.

  0 : (63 rows x 6 cols)


C:\Users\DELL\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [27]:
FF_5.head(5)

,Mkt-RF,SMB,HML,RMW,CMA,RF,Port_Ret,RP-Rf
Date,,,,,,,,
2022-03-01,-0.0156,-0.0028,-0.0083,-0.0037,0.0059,0.0,-0.019785,-0.019785
2022-03-02,0.0181,0.0062,0.0133,0.0047,0.0042,0.0,0.010955,0.010955
2022-03-03,-0.0088,-0.0021,0.0151,0.0124,0.0120,0.0,0.001506,0.001506
2022-03-04,-0.0110,-0.0049,0.0068,0.0041,0.0121,0.0,-0.009954,-0.009954
2022-03-07,-0.0312,0.0071,0.0090,-0.0069,0.0136,0.0,-0.031365,-0.031365


In [28]:
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                  RP-Rf   R-squared:                       0.841
Model:                            OLS   Adj. R-squared:                  0.827
Method:                 Least Squares   F-statistic:                     60.18
Date:                Mon, 04 Jul 2022   Prob (F-statistic):           1.73e-21
Time:                        16:46:28   Log-Likelihood:                 241.12
No. Observations:                  63   AIC:                            -470.2
Df Residuals:                      57   BIC:                            -457.4
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0004      0.001     -0.529      0.5